In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install sklearn.preprocessing

In [6]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.8 MB/s eta 0:00:00


In [7]:
!pip install transformers

In [8]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from deap import base, creator, tools, algorithms
from transformers import pipeline
import random
from random import sample, uniform, randint, choice
import requests
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import time
from collections import defaultdict
import math

In [9]:
file_path1 = '/content/drive/MyDrive/sqlite/synchronize.xlsx'
file_path2 = '/content/drive/MyDrive/sqlite/recipes_instructions_grouped.xlsx'

In [10]:
synchronize_df = pd.read_excel(file_path1)
recipes_instructions_df = pd.read_excel(file_path2)

In [11]:
synchronize_df

,Recipe Name,Ingredient Group,Ingredient Name,Amount,Unit,ingredient,processed,preliminary_processed,fresh,frozen,...,protein,carbohydrate,fat,Spicy,Bitter,Sweet,Sour,Savory,Step,Instructions
0,Kale Pesto Basil Wraps,Green Wrap,"(100g) chopped kale, stems removed",4,cups,chopped kale,False,True,False,False,...,2.92000,4.42000,1.49000,False,True,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
1,Kale Pesto Basil Wraps,Green Wrap,(50g) fresh basil leaves,2,cups,fresh basil leaves,False,False,True,False,...,1.57500,1.32500,0.32000,False,False,False,False,True,Step 1,"In a high-speed blender, combine the first ten..."
2,Kale Pesto Basil Wraps,Green Wrap,"jalapeños (about 28g), seeds removed",2,NaN,jalapeños,False,False,False,False,...,0.23442,1.67440,0.09531,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
3,Kale Pesto Basil Wraps,Green Wrap,(170g) chopped zucchini,1,cup,chopped zucchini,False,True,False,False,...,2.05700,5.28700,0.54400,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
4,Kale Pesto Basil Wraps,Green Wrap,celery ribs (about 80g),2,NaN,celery ribs,False,False,False,False,...,0.49128,2.11464,0.12104,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Roasted Vegetable Lasagna,NaN,onion powder,1,tsp,onion powder,True,False,False,False,...,0.24960,1.89840,0.02496,True,False,False,False,True,Step 4,Transfer the roasted cauliflower to a food pro...
1232,Roasted Vegetable Lasagna,NaN,red pepper flakes,0.25,tsp,red pepper flakes,True,False,False,False,...,0.02043,0.13470,0.01120,True,False,False,False,False,Step 4,Transfer the roasted cauliflower to a food pro...
1233,Roasted Vegetable Lasagna,NaN,ground black pepper,0.25,tsp,ground black pepper,True,False,False,False,...,0.05980,0.36800,0.01874,True,False,False,False,False,Step 4,Transfer the roasted cauliflower to a food pro...
1234,Roasted Vegetable Lasagna,NaN,jarred or homemade marinara sauce,3,cups,jarred or homemade marinara sauce,True,False,False,False,...,11.00880,58.84560,12.75120,False,False,False,True,True,Step 5,"To assemble, spread a layer of marinara sauce ..."


In [12]:
# Chuyển đổi cột 'Amount' từ object sang kiểu số, các giá trị không hợp lệ sẽ trở thành NaN
synchronize_df['Amount'] = pd.to_numeric(synchronize_df['Amount'], errors='coerce')

In [13]:
synchronize_df

,Recipe Name,Ingredient Group,Ingredient Name,Amount,Unit,ingredient,processed,preliminary_processed,fresh,frozen,...,protein,carbohydrate,fat,Spicy,Bitter,Sweet,Sour,Savory,Step,Instructions
0,Kale Pesto Basil Wraps,Green Wrap,"(100g) chopped kale, stems removed",4.00,cups,chopped kale,False,True,False,False,...,2.92000,4.42000,1.49000,False,True,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
1,Kale Pesto Basil Wraps,Green Wrap,(50g) fresh basil leaves,2.00,cups,fresh basil leaves,False,False,True,False,...,1.57500,1.32500,0.32000,False,False,False,False,True,Step 1,"In a high-speed blender, combine the first ten..."
2,Kale Pesto Basil Wraps,Green Wrap,"jalapeños (about 28g), seeds removed",2.00,NaN,jalapeños,False,False,False,False,...,0.23442,1.67440,0.09531,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
3,Kale Pesto Basil Wraps,Green Wrap,(170g) chopped zucchini,1.00,cup,chopped zucchini,False,True,False,False,...,2.05700,5.28700,0.54400,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
4,Kale Pesto Basil Wraps,Green Wrap,celery ribs (about 80g),2.00,NaN,celery ribs,False,False,False,False,...,0.49128,2.11464,0.12104,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Roasted Vegetable Lasagna,NaN,onion powder,1.00,tsp,onion powder,True,False,False,False,...,0.24960,1.89840,0.02496,True,False,False,False,True,Step 4,Transfer the roasted cauliflower to a food pro...
1232,Roasted Vegetable Lasagna,NaN,red pepper flakes,0.25,tsp,red pepper flakes,True,False,False,False,...,0.02043,0.13470,0.01120,True,False,False,False,False,Step 4,Transfer the roasted cauliflower to a food pro...
1233,Roasted Vegetable Lasagna,NaN,ground black pepper,0.25,tsp,ground black pepper,True,False,False,False,...,0.05980,0.36800,0.01874,True,False,False,False,False,Step 4,Transfer the roasted cauliflower to a food pro...
1234,Roasted Vegetable Lasagna,NaN,jarred or homemade marinara sauce,3.00,cups,jarred or homemade marinara sauce,True,False,False,False,...,11.00880,58.84560,12.75120,False,False,False,True,True,Step 5,"To assemble, spread a layer of marinara sauce ..."


In [14]:
herbs_and_spices_keywords = [
    "basil", "jalapeno", "garlic", "black pepper", "cumin", "oregano", "cilantro", "parsley", "paprika", "ancho chile",
    "lemon pepper", "zest of 1 orange", "lemon zest", "powder", "ground", "mustard", "chile", "asafoetida", "fenugreek",
    "cayenne", "garam masala", "lime zest", "ginger", "crushed red pepper", "red pepper flakes", "dill", "cinnamon",
    "chili", "thyme", "turmeric", "mint", "curry", "italian seasoning", "vanilla", "pumpkin pie spice", "sage", "bay",
    "celery seeds", "rosemary", "serrano", "coriander seeds", "lemongrass", "spices", "vinegar", "yeast", "miso",
    "Savory Spice Blend", "tahini", "Super-Charged Spice Blend", "Umami Sauce", "syrup", "blackstrap molasses",
    "Ranch Dressing", "Balsamic Date Glaze", "chlorella", "marinara"
]

In [15]:
pattern = re.compile(r'\b(' + '|'.join(re.escape(word) for word in herbs_and_spices_keywords) + r')\b', re.IGNORECASE)

In [16]:
# Load JSON Data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]

action_data = load_json_data('/content/drive/MyDrive/sqlite/Action.json')
food_entity_data = load_json_data('/content/drive/MyDrive/sqlite/Food_Entity.json')
process_data = load_json_data('/content/drive/MyDrive/sqlite/Process.json')

In [17]:
synchronize_df['Herbs and Spices'] = synchronize_df['Ingredient Name'].apply(lambda x: bool(pattern.search(str(x))))

In [18]:
def add_servings_to_synchronize(synchronize_df, recipes_instructions_df):
    # Tạo một từ điển để tra cứu các giá trị 'Servings' từ `recipes_instructions_df`
    recipe_servings = recipes_instructions_df[['Recipe Name', 'Servings']].drop_duplicates('Recipe Name').set_index('Recipe Name')['Servings'].to_dict()

    # Thêm cột 'serving' vào `synchronize_df` dựa trên từ điển 'recipe_servings'
    synchronize_df['serving'] = synchronize_df['Recipe Name'].map(recipe_servings)

    return synchronize_df

In [19]:
synchronize_df = add_servings_to_synchronize(synchronize_df, recipes_instructions_df)

In [20]:
synchronize_df

,Recipe Name,Ingredient Group,Ingredient Name,Amount,Unit,ingredient,processed,preliminary_processed,fresh,frozen,...,fat,Spicy,Bitter,Sweet,Sour,Savory,Step,Instructions,Herbs and Spices,serving
0,Kale Pesto Basil Wraps,Green Wrap,"(100g) chopped kale, stems removed",4.00,cups,chopped kale,False,True,False,False,...,1.49000,False,True,False,False,False,Step 1,"In a high-speed blender, combine the first ten...",False,2.0
1,Kale Pesto Basil Wraps,Green Wrap,(50g) fresh basil leaves,2.00,cups,fresh basil leaves,False,False,True,False,...,0.32000,False,False,False,False,True,Step 1,"In a high-speed blender, combine the first ten...",True,2.0
2,Kale Pesto Basil Wraps,Green Wrap,"jalapeños (about 28g), seeds removed",2.00,NaN,jalapeños,False,False,False,False,...,0.09531,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten...",False,2.0
3,Kale Pesto Basil Wraps,Green Wrap,(170g) chopped zucchini,1.00,cup,chopped zucchini,False,True,False,False,...,0.54400,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten...",False,2.0
4,Kale Pesto Basil Wraps,Green Wrap,celery ribs (about 80g),2.00,NaN,celery ribs,False,False,False,False,...,0.12104,False,False,False,False,False,Step 1,"In a high-speed blender, combine the first ten...",False,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Roasted Vegetable Lasagna,NaN,onion powder,1.00,tsp,onion powder,True,False,False,False,...,0.02496,True,False,False,False,True,Step 4,Transfer the roasted cauliflower to a food pro...,True,6.0
1232,Roasted Vegetable Lasagna,NaN,red pepper flakes,0.25,tsp,red pepper flakes,True,False,False,False,...,0.01120,True,False,False,False,False,Step 4,Transfer the roasted cauliflower to a food pro...,True,6.0
1233,Roasted Vegetable Lasagna,NaN,ground black pepper,0.25,tsp,ground black pepper,True,False,False,False,...,0.01874,True,False,False,False,False,Step 4,Transfer the roasted cauliflower to a food pro...,True,6.0
1234,Roasted Vegetable Lasagna,NaN,jarred or homemade marinara sauce,3.00,cups,jarred or homemade marinara sauce,True,False,False,False,...,12.75120,False,False,False,True,True,Step 5,"To assemble, spread a layer of marinara sauce ...",True,6.0


In [21]:
print(synchronize_df['Amount'].dtype)

float64


In [22]:
# Sử dụng mô hình nhận diện thực thể (NER)
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
# Sử dụng mô hình NLP để phân loại các bước
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [24]:
def get_nutrition(ingredients):
    """Lấy thông tin dinh dưỡng sử dụng API Edamam."""
    APP_ID = 'dfc9551d'
    APP_KEY = '4086f5c1929cda0a712d09af7f377da9'
    url = 'https://api.edamam.com/api/nutrition-details'

    headers = {'Content-Type': 'application/json'}
    data = {"title": "Generated Recipe", "ingr": ingredients}

    try:
        response = requests.post(f'{url}?app_id={APP_ID}&app_key={APP_KEY}', headers=headers, json=data)
        response.raise_for_status()  # Kiểm tra xem yêu cầu có thành công không
        nutrition_data = response.json()

        # Trích xuất thông tin dinh dưỡng nếu có
        calories = nutrition_data.get('calories', 'Không có thông tin')
        protein = nutrition_data.get('totalNutrients', {}).get('PROCNT', {}).get('quantity', 'Không có thông tin')
        carbs = nutrition_data.get('totalNutrients', {}).get('CHOCDF', {}).get('quantity', 'Không có thông tin')
        fat = nutrition_data.get('totalNutrients', {}).get('FAT', {}).get('quantity', 'Không có thông tin')

        # Đơn vị của mỗi giá trị (g hoặc kcal)
        protein_unit = nutrition_data.get('totalNutrients', {}).get('PROCNT', {}).get('unit', 'g')
        carbs_unit = nutrition_data.get('totalNutrients', {}).get('CHOCDF', {}).get('unit', 'g')
        fat_unit = nutrition_data.get('totalNutrients', {}).get('FAT', {}).get('unit', 'g')

        # In thông tin dinh dưỡng
        print("\n*** Thông tin dinh dưỡng ***")
        print(f"Calories: {calories} kcal")
        print(f"Protein: {protein} {protein_unit}")
        print(f"Carbs: {carbs} {carbs_unit}")
        print(f"Fat: {fat} {fat_unit}")

        return nutrition_data
    except requests.exceptions.HTTPError as http_err:
        print(f"Lỗi HTTP khi gọi API: {http_err}")
        print(f"Nội dung phản hồi từ server: {response.text}")
    except requests.exceptions.RequestException as req_err:
        print(f"Lỗi khi gọi API: {req_err}")
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return {}

In [25]:
def compute_average_amount(ingredient, target_servings):
  """Compute average amount for ingredients based on servings."""
  recipes_with_ingredient = synchronize_df[synchronize_df['Ingredient Name'] == ingredient]

  if recipes_with_ingredient.empty or recipes_with_ingredient['Amount'].isnull().all():
    return 1

  avg_amount = recipes_with_ingredient['Amount'].dropna().mean()
  avg_servings = recipes_with_ingredient['serving'].dropna().mean() if not recipes_with_ingredient['serving'].isnull().all() else 1

  if avg_servings > 0:
    return (avg_amount / avg_servings) * target_servings

  return 1

In [26]:
# Khởi tạo môi trường GA cho Ingredients
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [27]:
# Genetic Algorithm Functions
# Hàm tạo cá thể cho GA cho Ingredients
def create_individual(user_ingredients, herbs_spices):
  base_ingredients = set(user_ingredients.copy())
  additional_herbs_spices = sample([herb for herb in herbs_spices if herb not in base_ingredients], randint(2, 5))
  return list(base_ingredients.union(additional_herbs_spices))


In [28]:
#Đột biến trên hương vị
def flavor_based_mutation(individual, flavor_preferences, herbs_spices, user_ingredients, indpb=0.1):
  for i in range(len(individual)):
    if individual[i] not in user_ingredients and random.random() < indpb:
      possible_choices = [herb for herb in herbs_spices if herb not in individual]
      selected_flavor = random.choice(flavor_preferences['more'])
      filtered_choices = [choice for choice in possible_choices if synchronize_df.loc[synchronize_df['Ingredient Name'] == choice, selected_flavor].any()]
      if filtered_choices:
        individual[i] = random.choice(filtered_choices)
      else:
        individual[i] = random.choice(possible_choices)

  individual = creator.Individual(dict.fromkeys(individual))
  return individual,

In [29]:
def custom_crossover(ind1, ind2, user_ingredients, swap_prob=0.5):
    """Hàm crossover tùy chỉnh sử dụng phương pháp Blend nhưng bảo vệ nguyên liệu của người dùng."""
    min_length = min(len(ind1), len(ind2))

    for i in range(min_length):
        # Chỉ hoán đổi nếu gene không phải là nguyên liệu của người dùng
        if ind1[i] not in user_ingredients and ind2[i] not in user_ingredients:
            # Với xác suất `swap_prob`, hoán đổi các gene giữa ind1 và ind2
            if random.random() < swap_prob:
                ind1[i], ind2[i] = ind2[i], ind1[i]

    # Đảm bảo các cá thể trả về là dạng `creator.Individual`
    ind1 = creator.Individual(dict.fromkeys(ind1))
    ind2 = creator.Individual(dict.fromkeys(ind2))

    return ind1, ind2

In [30]:
def evaluate_individual(individual, flavor_preferences, user_ingredients, diversity_bonus=2):
    """Evaluate the fitness of an individual based on user preferences."""
    score = 0
    preferred_weight = 6
    disliked_weight = -5
    user_ingredient_weight = 8

    for ingredient in user_ingredients:
      if ingredient in individual:
        score += user_ingredient_weight

    for ingredient in individual:
        # Evaluate if the ingredient matches the preferred flavors
        for flavor in flavor_preferences['more']:
            if synchronize_df.loc[synchronize_df['Ingredient Name'] == ingredient, flavor].any():
                score += preferred_weight
        # Penalize if the ingredient matches the disliked flavors
        for flavor in flavor_preferences['less']:
            if synchronize_df.loc[synchronize_df['Ingredient Name'] == ingredient, flavor].any():
                score += disliked_weight

    unrelated_ingredients = set(individual) - set(user_ingredients)
    if len(unrelated_ingredients) > len(user_ingredients) / 2:
      score -= 10

    # Bonus điểm dựa trên độ đa dạng của nguyên liệu
    unique_herbs_spices = set([i for i in individual if i in herbs_and_spices_keywords])
    score += len(unique_herbs_spices) * diversity_bonus

    return score,


In [31]:
def update_population_fitness_with_sharing(population, flavor_preferences, user_ingredients, sharing_radius=3.0):
    """Update fitness of the population using Fitness Sharing."""
    for ind in population:
        base_fitness = evaluate_individual(ind, flavor_preferences, user_ingredients)[0]
        shared_fitness = base_fitness

        for other in population:
            if other != ind:
                distance = len(set(ind) ^ set(other))  # Đo sự khác biệt giữa hai cá thể
                if distance < sharing_radius:
                    # Giảm điểm fitness nếu khoảng cách nhỏ hơn sharing_radius
                    shared_fitness *= (1 - (sharing_radius - distance) / sharing_radius)

        ind.fitness.values = (shared_fitness,)

In [32]:
def adaptive_mutation_rate(population, current_gen, max_gen):
    """Điều chỉnh tỷ lệ đột biến dựa trên mức độ hội tụ của quần thể."""
    diversity = calculate_population_diversity(population)
    base_mutation_rate = 0.1  # Tỷ lệ đột biến cơ bản
    max_mutation_rate = 0.5   # Tỷ lệ đột biến tối đa

    # Nếu đa dạng quần thể thấp, tăng tỷ lệ đột biến
    if diversity < 0.3:  # Ngưỡng đa dạng thấp
        adaptive_mutation = base_mutation_rate + (max_mutation_rate - base_mutation_rate) * (1 - (current_gen / max_gen))
    else:
        adaptive_mutation = base_mutation_rate

    if diversity < 0.1:
        adaptive_mutation += 0.3
    return min(adaptive_mutation, max_mutation_rate)  # Giới hạn tỷ lệ đột biến

In [33]:
def merge_ingredient_variations(ingredients_list):
    """Hợp nhất các biến thể nguyên liệu trùng lặp và cộng số lượng, không lặp mô tả."""
    # Sử dụng defaultdict để lưu trữ thông tin về mỗi nguyên liệu
    ingredient_dict = defaultdict(lambda: {"amount": 0, "unit": "", "description": set()})

    for ingredient in ingredients_list:
        base_name = ingredient['ingredient'].lower()
        amount = ingredient['amount']
        unit = ingredient['unit']
        description = ingredient.get('description', base_name)  # Dùng tên cơ bản nếu không có mô tả

        # Cộng dồn số lượng cho nguyên liệu trùng tên
        ingredient_dict[base_name]["amount"] += amount
        ingredient_dict[base_name]["unit"] = unit if ingredient_dict[base_name]["unit"] == "" else ingredient_dict[base_name]["unit"]
        ingredient_dict[base_name]["description"].add(description)

    # Kết hợp lại thành danh sách tối ưu
    merged_ingredients = []
    for base_name, details in ingredient_dict.items():
        merged_description = ", ".join(sorted(details["description"]))
        merged_ingredient = f"{details['amount']} {details['unit']} {merged_description}".strip()
        merged_ingredients.append(merged_ingredient)

    return merged_ingredients


In [34]:
# Hàm rank-based selection mới
def rank_based_selection(population, k, elitism_rate=0.15):
    """Rank-based Selection with Elitism."""
    # Xếp hạng các cá thể dựa trên fitness của chúng (từ cao xuống thấp)
    ranked_population = sorted(population, key=lambda ind: ind.fitness.values[0], reverse=True)

    # Giữ lại các cá thể tốt nhất dựa trên elitism rate
    elite_count = int(len(population) * elitism_rate)
    elite_individuals = ranked_population[:elite_count]

    # Tính tổng số thứ hạng
    total_ranks = sum(range(1, len(ranked_population) + 1))

    # Xác suất chọn dựa trên thứ hạng
    probabilities = [(len(ranked_population) - rank + 1) / total_ranks for rank in range(1, len(ranked_population) + 1)]

    # Chọn cá thể ngẫu nhiên dựa trên xác suất từ thứ hạng
    selected_individuals = random.choices(ranked_population, weights=probabilities, k=k - elite_count)

    # Kết hợp các cá thể elite với các cá thể được chọn
    return elite_individuals + selected_individuals

In [35]:
def calculate_population_diversity(population):
    """Tính độ đa dạng của quần thể dựa trên độ khác biệt giữa các cá thể."""
    unique_individuals = set(tuple(ind) for ind in population)
    diversity_ratio = len(unique_individuals) / len(population)

    differences = [len(set(ind1) ^ set(ind2)) for ind1 in population for ind2 in population if ind1 != ind2]
    avg_difference = np.mean(differences) if differences else 0

    return diversity_ratio

In [53]:
def genetic_algorithm_ingredients(user_ingredients, herbs_spices, flavor_preferences, ngen=50, pop_size=150, cxpb=0.6, mutpb=0.2):
  toolbox = base.Toolbox()
  toolbox.register("individual", tools.initIterate, creator.Individual, lambda: create_individual(user_ingredients, herbs_spices))
  toolbox.register("population", tools.initRepeat, list, toolbox.individual)
  toolbox.register("mate", custom_crossover, user_ingredients=user_ingredients, swap_prob=0.5)
  toolbox.register("mutate", flavor_based_mutation, flavor_preferences=flavor_preferences, herbs_spices=herbs_spices, user_ingredients=user_ingredients, indpb=0.2)
  toolbox.register("select", rank_based_selection)
  toolbox.register("evaluate", lambda ind: evaluate_individual(ind, flavor_preferences, user_ingredients, diversity_bonus=2))

  population = toolbox.population(n=pop_size)
  invalid_ind = [ind for ind in population if not ind.fitness.valid]
  update_population_fitness_with_sharing(invalid_ind, flavor_preferences, user_ingredients, sharing_radius=3.0)

  hof = tools.HallOfFame(10)
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("min", np.min)
  stats.register("max", np.max)

  for gen in range(ngen):
    mutpb = adaptive_mutation_rate(population, gen, ngen)
    offspring = toolbox.select(population, len(population) - len(hof))
    offspring = list(map(toolbox.clone, offspring))

    # Lai ghép và đột biến
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < cxpb:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    for mutant in offspring:
      if random.random() < mutpb:
        if random.random() < 0.5:
          toolbox.mutate(mutant)
        del mutant.fitness.values

    # Đánh giá lại các cá thể con sau khi lai ghép và đột biến
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    update_population_fitness_with_sharing(invalid_ind, flavor_preferences, user_ingredients, sharing_radius=2.0)

    # Cập nhật quần thể với các cá thể tốt nhất từ thế hệ trước
    population[:] = offspring + list(hof)

    # Cập nhật Elitism
    hof.update(population)

    # Thu thập số liệu thống kê
    record = stats.compile(population)
    print(f"Generation {gen}: {record}")

    # Điều chỉnh tỷ lệ đột biến dựa trên sự đa dạng sau mỗi 10 thế hệ
    if gen % 10 == 0:
      diversity = calculate_population_diversity(population)
      if diversity < 0.2:
        mutpb = min(mutpb + 0.05, 0.5)
      else:
        mutpb = max(mutpb - 0.05, 0.1)

  # Trả về cá thể tốt nhất từ Hall of Fame
  best_individual = tools.selBest(population, 1)[0]
  return best_individual

In [54]:
from difflib import SequenceMatcher

def are_steps_similar(step1, step2, threshold=0.8):
    """Kiểm tra nếu hai bước hướng dẫn tương tự nhau dựa trên một ngưỡng."""
    similarity = SequenceMatcher(None, step1, step2).ratio()
    return similarity >= threshold

In [55]:
def adjust_fitness(population):
    """Điều chỉnh fitness để không có giá trị âm khi sử dụng selRoulette."""
    min_fitness = min(ind.fitness.values[0] for ind in population)
    if min_fitness < 0:
        adjustment = abs(min_fitness) + 1  # Thêm lượng bù dương
        for ind in population:
            ind.fitness.values = (ind.fitness.values[0] + adjustment,)

In [56]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

def refine_instructions_with_model(steps):
    """Sử dụng mô hình NLP để làm sáng tỏ và tóm tắt hướng dẫn."""
    refined_steps = []
    for step in steps:
        input_ids = tokenizer.encode(f"summarize: {step}", return_tensors="pt")
        summary_ids = model.generate(input_ids, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        refined_steps.append(summary)

    return refined_steps

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [57]:
# Genetic Algorithm for Directions
creator.create("FitnessDirection", base.Fitness, weights=(1.0,))
creator.create("DirectionIndividual", list, fitness=creator.FitnessDirection)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessDirection' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'DirectionIndividual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [58]:
def remove_duplicates(directions):
    """Remove duplicate steps from directions."""
    seen = set()
    unique_directions = []
    for step in directions:
        if not any(are_steps_similar(step, seen_step) for seen_step in seen):
            unique_directions.append(step)
            seen.add(step)
    return unique_directions

In [59]:
# Direction Handling and Genetic Algorithm
def create_direction_individual(all_steps, threshold=0.8):
  """Create an individual for recipe directions."""
  individual = []
  while len(individual) < randint(5,8):
    new_step = random.choice(all_steps)
    if not any(are_steps_similar(new_step, existing_step, threshold) for existing_step in individual):
      individual.append(new_step)
  individual = remove_duplicates(individual)
  return individual

In [60]:
def categorize_steps_with_nlp(steps):
      """Phân loại và sắp xếp các bước nấu ăn theo thứ tự hợp lý."""
      categories = {'preparation': [], 'cooking': [], 'finishing': []}
      for step in steps:
        # Phân loại bước dựa trên mô hình NLP
        prediction = classifier(step)[0]['label'].lower()

        if 'prep' in prediction:
          categories['preparation'].append(step)
        elif 'cook' in prediction:
          categories['cooking'].append(step)
        elif 'finish' in prediction:
          categories['finishing'].append(step)
        else:
          categories['cooking'].append(step)

        # Sắp xếp theo thứ tự: Chuẩn bị -> Nấu chính -> Hoàn thiện
      sorted_directions = categories['preparation'] + categories['cooking'] + categories['finishing']
      return sorted_directions

In [61]:
def filter_directions_by_ingredients_nlp(directions, relevant_ingredients):
      """Lọc các bước hướng dẫn dựa trên các nguyên liệu chính."""
      filtered_directions = []

      for step in directions:
        entities = ner(step)
        detected_ingredients = [entity['word'].lower() for entity in entities if entity['entity'] == 'B-INGREDIENT']

        if sum(1 for ingredient in relevant_ingredients if ingredient in detected_ingredients) >= len(relevant_ingredients) / 2:
          filtered_directions.append(step)

      return filtered_directions

In [62]:
def extract_relevant_processes(relevant_ingredients, process_data):
      """Extract relevant processes for evaluation based on selected ingredients."""
      relevant_process = {}
      for entry in process_data:
        if isinstance(entry, dict):
            recipe_name = list(entry.keys())[0]
            steps = entry[recipe_name]

            processes = []

            if isinstance(steps, dict):
                for step_key, step_data in steps.items():
                    if isinstance(step_data, list):
                        processes.extend(step_data)
                    else:
                        processes.append(step_data)
            relevant_process[recipe_name] = processes
      return relevant_process

In [63]:
def evaluate_direction_individual(individual, relevant_ingredients, synchronize_df, process_data):
        """Evaluate the fitness of an individual direction."""
        unique_individual = remove_duplicates(individual)
        filtered_steps = filter_directions_by_ingredients_nlp(unique_individual, relevant_ingredients)
        sorted_steps = categorize_steps_with_nlp(filtered_steps)
        score = 0
        unique_steps = set(sorted_steps)  # Sử dụng tập hợp để kiểm tra trùng lặp
        # Nếu số lượng các bước duy nhất nhỏ hơn số bước tổng cộng, phạt điểm
        if len(unique_steps) < len(sorted_steps):
          score -= (len(sorted_steps) - len(unique_steps)) * 3  # Phạt cho mỗi bước trùng lặp

        covered_ingredients = set()
        for step in sorted_steps:
          for ingredient in relevant_ingredients:
            if ingredient in step:
              covered_ingredients.add(ingredient)
              score +=4

        if len(covered_ingredients) < len(relevant_ingredients) / 2:
          score -= 3

        previous_step = None
        for step in unique_individual:
          if previous_step and are_steps_similar(previous_step, step):
            score -= 2
          previous_step = step

        return score,

In [64]:
 # Extract relevant steps based on ingredients
def extract_relevant_steps(ingredients, synchronize_df, recipes_instructions_df, food_entity_data):
        """Extract relevant steps for cooking based on the selected ingredients."""
        all_steps = []
        food_entities = set()

        for entry in food_entity_data:
          recipe_name = entry.get("Recipe Name", "")
          entities = entry.get("Food Entity", [])
          food_entities.update(entities)

        for ingredient in ingredients:
            related_steps = synchronize_df[synchronize_df['Ingredient Name'] == ingredient]
            related_steps = related_steps[['Step', 'Instructions', 'Recipe Name']].dropna(subset=['Step', 'Instructions'])

            if related_steps.empty:
              continue

            all_steps.extend(related_steps['Instructions'].tolist())

            related_recipes = related_steps['Recipe Name'].unique()

            for recipe_name in related_recipes:
              full_recipe_instructions = recipes_instructions_df[recipes_instructions_df['Recipe Name'] == recipe_name][['Step', 'Instructions']].dropna()

              filtered_instructions = []
              for _, row in full_recipe_instructions.iterrows():
                step = row['Instructions']
                words = step.split()

                contains_relevant_ingredient = any(ingredient in step for ingredient in ingredients)

                contains_unrelated_entity = any(word in food_entities and word not in ingredients for word in words)

                if not contains_unrelated_entity:
                  filtered_instructions.append(step)

              all_steps.extend(filtered_instructions)

        return all_steps

In [65]:
def two_point_crossover(ind1, ind2):
    """Thực hiện two-point crossover."""
    if len(ind1) < 3 or len(ind2) < 3:
        return ind1, ind2  # Nếu cá thể quá ngắn, không thực hiện crossover

    # Chọn hai điểm crossover ngẫu nhiên
    point1 = random.randint(1, min(len(ind1), len(ind2)) - 2)
    point2 = random.randint(point1 + 1, min(len(ind1), len(ind2)) - 1)

    # Tạo ra hai con bằng cách trao đổi phần giữa hai điểm crossover
    child1 = ind1[:point1] + ind2[point1:point2] + ind1[point2:]
    child2 = ind2[:point1] + ind1[point1:point2] + ind2[point2:]

    return creator.DirectionIndividual(child1), creator.DirectionIndividual(child2)

In [66]:
def update_population_fitness_with_sharing_directions(population, relevant_ingredients, synchronize_df, process_data, sharing_radius=2.0):
    """Update fitness of the population in GA directions using Fitness Sharing."""
    for ind in population:
        # Tính fitness cơ bản cho từng cá thể
        base_fitness = evaluate_direction_individual(ind, relevant_ingredients, synchronize_df, process_data)[0]
        shared_fitness = base_fitness

        # Điều chỉnh fitness dựa trên sự tương đồng của cá thể với các cá thể khác trong quần thể
        for other in population:
            if other != ind:
                # Tính khoảng cách giữa các cá thể dựa trên số bước khác biệt
                distance = len(set(ind) ^ set(other))  # Đo sự khác biệt giữa hai cá thể
                if distance < sharing_radius:
                    # Giảm điểm fitness nếu khoảng cách nhỏ hơn sharing_radius
                    shared_fitness *= (1 - (sharing_radius - distance) / sharing_radius)

        # Cập nhật lại giá trị fitness đã được điều chỉnh
        ind.fitness.values = (shared_fitness,)

In [67]:
def genetic_algorithm_directions(all_relevant_steps, relevant_ingredients, ngen=10, pop_size=30, cxpb=0.6, mutpb=0.2, threshold=0.85):
        """Chạy thuật toán GA để tối ưu hóa hướng dẫn nấu ăn."""
        toolbox_directions = base.Toolbox()
        toolbox_directions.register("individual", tools.initIterate, creator.DirectionIndividual,
                                     lambda: create_direction_individual(all_relevant_steps, threshold=0.85))
        toolbox_directions.register("population", tools.initRepeat, list, toolbox_directions.individual)
        toolbox_directions.register("mate", two_point_crossover)
        toolbox_directions.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
        toolbox_directions.register("select", tools.selRoulette)
        toolbox_directions.register("evaluate", lambda ind: evaluate_direction_individual(ind, relevant_ingredients, synchronize_df, process_data))

        population = toolbox_directions.population(n=pop_size)
        invalid_ind = [ind for ind in population if not ind.fitness.valid]
        update_population_fitness_with_sharing_directions(invalid_ind, relevant_ingredients, synchronize_df, process_data, sharing_radius=3.0)
        adjust_fitness(invalid_ind)

        hof = tools.HallOfFame(10)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("min", np.min)
        stats.register("max", np.max)

        # Bắt đầu các thế hệ GA
        for gen in range(ngen):
          mutpb = adaptive_mutation_rate(population, gen, ngen)

          # Chọn các cá thể con từ quần thể hiện tại bằng Roulette Selection
          offspring = toolbox_directions.select(population, len(population) - len(hof))
          offspring = list(map(toolbox_directions.clone, offspring))

          # Lai ghép (crossover) và đột biến (mutation)
          for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
              toolbox_directions.mate(child1, child2)
              del child1.fitness.values
              del child2.fitness.values
          for mutant in offspring:
            if random.random() < mutpb:
              toolbox_directions.mutate(mutant)
              del mutant.fitness.values

          # Đánh giá lại các cá thể không hợp lệ sau đột biến và lai ghép
          invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
          if gen % 10 == 0 and invalid_ind:
            update_population_fitness_with_sharing_directions(invalid_ind, relevant_ingredients, synchronize_df, process_data, sharing_radius=3.0)
            adjust_fitness(invalid_ind)  # Điều chỉnh fitness nếu cần thiết cho selRoulette

          # Kết hợp offspring với các cá thể elitist
          population[:] = tools.selBest(population + offspring, pop_size)

          # Cập nhật Elitism
          hof.update(population)

          # Thu thập số liệu thống kê cho thế hệ hiện tại
          record = stats.compile(population)
          print(f"Generation {gen}: {record}")

        best_directions = tools.selBest(population, 1)[0]
        best_directions = remove_duplicates(best_directions)
        return best_directions

In [72]:
# Hàm chính
def main():
    print("===ỨNG DỤNG TÌM KIẾM CÔNG THỨC NẤU ĂN VỚI NGUYÊN LIỆU CÓ SẴN CỦA BẠN")
    def suggest_user_ingredients():
      """Gợi ý một số nguyên liệu phổ biến nếu người dùng không biết chọn gì."""
      common_ingredients = ["broccoli", "cabbage", "tomato", "carrot", "potato", "red pepper", "berries", "lentils", "chickpeas"]
      print("Gợi ý nguyên liệu phổ biến:", ", ".join(common_ingredients))
    # Đọc dữ liệu từ người dùng
    while True:
      user_ingredients = input("Nhập ít nhất 3 nguyên liệu mà bạn muốn tạo món ăn (phân cách bởi dấu phẩy): ").split(',')
      user_ingredients = [i.strip() for i in user_ingredients if i.strip()]
      if len(user_ingredients) >= 3:
        break
      else:
        print("Bạn chưa nhập đủ 3 nguyên liệu. Vui lòng nhập lại.")
        suggest_user_ingredients()

    more_flavor = int(input("Bạn muốn hương vị nào là ưu tiên? (1. More Sweet, 2.More Bitter, 3. More Savory, 4. More Spicy, 5. More Sour): "))
    less_flavor = int(input("Bạn không thích hương vị nào? (1. Less Sweet, 2. Less Bitter, 3. Less Savory, 4. Less Spicy, 5. Less Sour): "))

    target_servings = int(input("Khẩu phần ăn của bạn? (1,2,3,4,5,6,7,8,9,10): "))

    # Lựa chọn hương vị ưu tiên và không thích
    flavors = ['Sweet', 'Bitter', 'Savory', 'Spicy', 'Sour']
    flavor_preferences = {
        'more': [flavors[more_flavor - 1]],
        'less': [flavors[less_flavor - 1]]
    }

    # Lấy danh sách các loại thảo mộc/gia vị từ bộ dữ liệu synchronize
    herbs_spices = synchronize_df[synchronize_df['Herbs and Spices'] == True]['Ingredient Name'].tolist()

    # Chạy GA để tối ưu hóa danh sách nguyên liệu
    best_ingredients = genetic_algorithm_ingredients(user_ingredients, herbs_spices, flavor_preferences, ngen=50, pop_size=150, cxpb=0.6, mutpb=0.2)

    optimized_ingredients = []
    for ingredient in best_ingredients:
        amount = round(compute_average_amount(ingredient, target_servings), 2)
        ingredient_unit_series = synchronize_df[synchronize_df['Ingredient Name'] == ingredient]['Unit']
        unit = ingredient_unit_series.iloc[0] if not ingredient_unit_series.empty else ' '

        if isinstance(unit, float) and math.isnan(unit):
            unit = ' '

        optimized_ingredients.append({
            'amount': amount,
            'unit': unit,
            'ingredient': ingredient
        })

    optimized_ingredients = merge_ingredient_variations(optimized_ingredients)
    for item in optimized_ingredients:
        print(item)

    print("\n*** Nguyên liệu tối ưu ***")
    for ingredient in optimized_ingredients:
        print(ingredient)

    # Tạo hướng dẫn nấu ăn bằng GA
    start_time = time.time()
    relevant_ingredients = best_ingredients
    print(f"Thời gian chạy: {time.time() - start_time} giây")

    start_time = time.time()
    all_relevant_steps = extract_relevant_steps(relevant_ingredients, synchronize_df, recipes_instructions_df, food_entity_data)
    print(f"Thời gian chạy ra các bước hướng dẫn liên quan: {time.time() - start_time} giây")

    start_time = time.time()
    best_directions = genetic_algorithm_directions(all_relevant_steps, relevant_ingredients, ngen=10, pop_size=30, cxpb=0.6, mutpb=0.1, threshold=0.85)
    print(f"Thời gian chạy best_directions: {time.time() - start_time} giây")

    #print("\n*** Hướng dẫn ***")
    #for i, step in enumerate(best_directions, 1):
    #    print(f"Bước {i}: {step}")

    start_time = time.time()
    refined_direction = refine_instructions_with_model(best_directions)
    print(f"Thời gian chạy: {time.time() - start_time} giây")

    # Hiển thị kết quả

    print("\n*** Nguyên liệu ***")
    for ingredient in optimized_ingredients:
        print(ingredient)

    print("\n*** Hướng dẫn ***")
    for i, step in enumerate(refined_direction, 1):
        print(f"Bước {i}: {step}")

    nutrition = get_nutrition(optimized_ingredients)


In [74]:
if __name__ == "__main__":
   main()

===ỨNG DỤNG TÌM KIẾM CÔNG THỨC NẤU ĂN VỚI NGUYÊN LIỆU CÓ SẴN CỦA BẠN
Nhập ít nhất 3 nguyên liệu mà bạn muốn tạo món ăn (phân cách bởi dấu phẩy): carrot,chickpeas,tomato,kale
Bạn muốn hương vị nào là ưu tiên? (1. More Sweet, 2.More Bitter, 3. More Savory, 4. More Spicy, 5. More Sour): 4
Bạn không thích hương vị nào? (1. Less Sweet, 2. Less Bitter, 3. Less Savory, 4. Less Spicy, 5. Less Sour): 1
Khẩu phần ăn của bạn? (1,2,3,4,5,6,7,8,9,10): 10
Generation 0: {'avg': 35.842222222222226, 'min': 12.0, 'max': 54.0}
Generation 1: {'avg': 40.913333333333334, 'min': 20.0, 'max': 54.0}
Generation 2: {'avg': 45.278333333333336, 'min': 6.75, 'max': 54.0}
Generation 3: {'avg': 49.14666666666667, 'min': 27.0, 'max': 56.0}
Generation 4: {'avg': 49.134739583333335, 'min': 0.2109375, 'max': 56.0}
Generation 5: {'avg': 51.189166666666665, 'min': 1.6875, 'max': 56.0}
Generation 6: {'avg': 51.85666666666667, 'min': 7.0, 'max': 56.0}
Generation 7: {'avg': 50.76083333333333, 'min': 3.375, 'max': 58.0}
Genera